In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

# get ESTA_API_KEY
api_key = os.environ.get('ESTA_API_KEY')

# set url
url = 'https://d09uj3e9kh.execute-api.us-east-1.amazonaws.com/dev/inference'

In [ ]:
import concurrent.futures
import csv
import time
import requests


def fetch_and_write(row):
    start_time = time.time()
    req = requests.get(
        url,
        params={
            "query": row[0],
            "collectionName": "levio-hr-new",
        },
        headers={
            "x-api-key": api_key,
            "accept": "application/json",
            "Content-Type": "application/json",
        },
    )
    time_taken = time.time() - start_time
    answer = req.json()
    return [row[0], answer.strip('"'), "{:.2f}".format(time_taken)]


with open("data/questions.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    rows = list(reader)

    with open(
        "data/questions_answers.csv", "w", encoding="utf-8", newline="\n"
    ) as outfile:
        writer = csv.writer(outfile, delimiter=";")
        writer.writerow(["Question", "Answer", "TTC"])  # Write header

        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = executor.map(fetch_and_write, rows)
            for result in results:
                writer.writerow(result)